# some needed code from part2

In [50]:
def data_tuple(file):

    data = []
    lst = []
    with open(file, "r") as f:
        for line in f:
            if line == "\n":
              data.append(lst)
              lst = []
            
            else:
              lines = line.replace("\n",'').split(" ")
              lst.append(tuple(lines))
            
    return data
def smoothing_em(data, k):

    new_data = []
    new_tweets = []
  
    word_count = {}
    
    for lst in data:
        for wordTuples in lst:
          
            word = wordTuples[0]
            tag = wordTuples[1]
            
            word_count[word] = word_count.get(word, 0) + 1

            if word_count[word] < k:
              
               new_tweets.append(("#UNK#",tag))
                
            else:
              
                new_tweets.append(wordTuples)
                
        new_data.append(new_tweets)
        new_tweets = []
        
    return new_data,word_count




def emission_parameter(data):
    
    tag_count = {}
    match_count = {}
    
    for lst in data:         
        for wordTuples in lst:
            
            tag = wordTuples[1]
            
            tag_count[tag] = tag_count.get(tag, 0) + 1
            match_count[wordTuples] = match_count.get(wordTuples, 0) + 1
            
      
    em = {i: match_count[i]/tag_count[i[1]] for i in match_count}   
    return tag_count, em


            
    

# part3

In [51]:
def create_transition_parameter(data):
    tag_transition_count={}
    #(tag0,tag1):count
    tag_count = {}
    #tag0:count
    for tweet in data:
        #data:list of (word,tag) tuples
        for i in range(len(tweet)-1):
            tag1 = tweet[i][1]
            if i ==0:
                tag0 = "START"
            else:
                tag0 = tweet[i-1][1]
            tag_count[tag0]=tag_count.get(tag0,0)+1
            tag_transition=(tag0,tag1)
            tag_transition_count[tag_transition] = tag_transition_count.get(tag_transition,0)+1
            if i == len(tweet)-1:
                tag0 = tag[1]
                tag[1] = "STOP"
                tag_count[tag0]=tag_count.get(tag0,0)+1
                tag_transition=(tag0,tag1)
                tag_transition_count[tag_transition] = tag_transition_count.get(tag_transition,0)+1
    transition_parameter = {k: tag_transition_count[k]/tag_count[k[0]]
                        for k in tag_transition_count}
    return transition_parameter

def create_start_parameter(data):
    start_tag_count={}
    #tag:count
    total_tweet_count =0
    for tweet in data:
        start_tag = tweet[0][1]
        start_tag_count[start_tag]=start_tag_count.get(start_tag,0)+1
        total_tweet_count += 1
    start_parameter = {k: start_tag_count[k]/total_tweet_count
                        for k in start_tag_count}
    return start_parameter

    
        
def viterbi(words, tags, start_p, trans_p, emit_p):
    V = [{}]
    path = {}
    for y in tags:
        V[0][y] = start_p.get(y,0) * emit_p.get((words[0],y),0)
        path[y] = [y]

    for t in range(1,len(words)):
        V.append({})
        newpath = {}

        for y in tags:
            (prob, tag) = max([(V[t-1][y0] * trans_p.get((y0,y),0) * emit_p.get((words[t],y),0), y0) 
                               for y0 in tags])
            V[t][y] = prob
            newpath[y] = path[tag] + [y]

        path = newpath
    (prob, tag) = max([(V[len(words) - 1][y], y) for y in tags])
    return (prob, path[tag])




def write_hmm_prediction(country, part, prediction_function,
                         word_sequence,tags, start_param, emit_param, trans_param):
    
    input_filename = country + "/dev.in"
    output_filename = country + "/dev.p"+part+".out"
    indata = []
    with open(input_filename, "r") as infile:
        indata = infile.read().strip('\n').split('\n\n') 
    
    with open(output_filename, "w") as outfile:
        for tweet in indata:
            word_sequence = tweet.split('\n')
            predicted_tag_sequence = prediction_function(word_sequence,
                                                tags, start_param, trans_param, emit_param)[1]
            if len(word_sequence) != len(predicted_tag_sequence):
                print("WARNING!! Different length {} / {}"\
                      .format(word_sequence, predicted_tag_sequence))
            for i in range(len(word_sequence)):
                line = "{} {}\n".format(word_sequence[i], 
                                        predicted_tag_sequence[i])
                outfile.write(line)
            outfile.write("\n")


from datetime import datetime
folders = ["CN (1)", "EN", "SG(1)", "FR"]
for folder in folders:
    start = datetime.now()
    data = data_tuple(folder+"/train")
    supress_data,words = smoothing_em(data,3)

    tags,emit_param = emission_parameter(supress_data)[0].keys(),emission_parameter(supress_data)[1]
    trans_param = create_transition_parameter(supress_data)
    start_param = create_start_parameter(supress_data)
    write_hmm_prediction(c,"3", viterbi,
                        words, tags, start_param, emit_param, trans_param)
    end = datetime.now()
    delt = end - start
    print("{} part 3 done in {}.{}s"\
          .format(c, delt.seconds, delt.microseconds))








EN part 3 done in 0.618115s
EN part 3 done in 0.446967s
EN part 3 done in 1.457448s
EN part 3 done in 0.235937s


# part5

In [60]:
#perceptron for part 5
from collections import defaultdict
import random, os


def read_unlabel_file(folder_path, filename):
    tweets = []
    with open(os.path.join(folder_path, filename), 'r', encoding="utf8") as infile:
        for line in infile:
            tweet = line.strip()
            if tweet!="":
                tweets.append(tweet)
            else:
                tweets.append("")
    return tweets


def read_label_file(folder_path, filename):
    tweets = []
    tag_counts = {}
    with open(os.path.join(folder_path, filename), 'r', encoding="utf8") as infile:
        for line in infile:
            if line.strip()!="":
                tweet = line.strip().split(" ")
                tweets.append((tweet[0],tweet[1]))
                tag_counts[tweet[1]] = tag_counts.get(tweet[1], 0) + 1
            else:
                tweets.append(("",""))
    return tweets,tag_counts

n=10
class perceptron():
    
    def __init__(self, tag_counts):
        self.tags = {}
        self.classes=tag_counts.keys()
        self.tag_counts = tag_counts
        


    def predict(self,feature):
        scores = defaultdict(float)
        if feature !="":
            if feature in self.tags:
                tags = self.tags[feature]
                for clas, tag in tags.items():
                    scores[clas] +=tag
            return max(self.classes, key=lambda clas: (scores[clas], clas))
        return ""

    def train(self, n_iter, examples):
        for i in range(n_iter):
            for features, ntag in examples:
                if features!="":
                    guess = self.predict(features)
                    if guess != ntag:
                        if features not in self.tags:
                            self.tags[features]={}
                        
                        self.tags[features][ntag] = self.tags[features].get(ntag,0) + 1
                        self.tags[features][guess] = self.tags[features].get(guess,0) - 1
                        
            random.shuffle(examples)



folders = ["FR","EN"]

for folder in folders:
    start = datetime.now()
    feature_pairs, weight_counts = read_label_file(folder, "train")
    test = perceptron(weight_counts)
    test.train(n, feature_pairs)
    with open(os.path.join(folder, "test.p5.out"), 'w', encoding="utf8") as outfile:
        for feature in read_unlabel_file(folder, "test.in"):
            outfile.write(feature + " " + test.predict(feature)+"\n")
    end = datetime.now()
    delt = end - start
    print("{} part 5 done in {}.{}s"\
          .format(folder, delt.seconds, delt.microseconds))


FR part 5 done in 1.339318s
EN part 5 done in 1.11680s
